aaa

In [38]:
# https://qiita.com/shimopino/items/f096cd92c5107ee02208
# https://qiita.com/ttttkkkkk31525/items/c2ee07f27065a03c56f2
# PGGANはProgressive Growing GANの略で、Progressive GANとも呼ばれます (GitHubの検索とかのときのために統一してほしい...)。画質を少しずつ向上させることによって、以前のものより高画質な画像を生成できるようになります。
# https://gist.github.com/bababax11/d8297befc4e02bba3b21b473ae92e58f
# https://www.inoue-kobo.com/ai_ml/gan-pikachu/index.html#%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AE%E6%A7%8B%E9%80%A0
# https://github.com/lhideki/gan-pikachu


In [39]:
# !pip install chromedriver-binary==112.0.5615.28.0
# !pip uninstall chromedriver-binary
# !pip install chromedriver-binary==112.0.5615.28.0

Found existing installation: chromedriver-binary 112.0.5615.28.0
Uninstalling chromedriver-binary-112.0.5615.28.0:
  Would remove:
    /Users/seijun/miniforge3/envs/python310/bin/chromedriver-path
    /Users/seijun/miniforge3/envs/python310/lib/python3.10/site-packages/chromedriver_binary-112.0.5615.28.0.dist-info/*
    /Users/seijun/miniforge3/envs/python310/lib/python3.10/site-packages/chromedriver_binary/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [58]:
# https://www.sukerou.com/2021/09/pythonselenium.html
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import os
from selenium import webdriver              # Webブラウザを自動操作する（python -m pip install selenium)
import chromedriver_binary                  # パスを通すためのコード
from selenium.webdriver.chrome.options import Options # オプションを使うために必要
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time


# option = Options()                          # オプションを用意
# option.add_argument('--incognito')          # シークレットモードの設定を付与
driver = webdriver.Chrome()
# driver = webdriver.Chrome(options=option)   # Chromeを準備(optionでシークレットモードにしている）

url = "https://zukan.pokemon.co.jp"

#ブラウザのウインドウ高を取得する
driver.get(url)
win_height = driver.execute_script("return window.innerHeight")

#スクロール開始位置の初期値（ページの先頭からスクロールを開始する）
last_top = 1

#ページの最下部までスクロールする無限ループ
while True:

  #スクロール前のページの高さを取得
  last_height = driver.execute_script("return document.body.scrollHeight")
  
  #スクロール開始位置を設定
  top = last_top

  #ページ最下部まで、徐々にスクロールしていく
  while top < last_height:
    top += int(win_height * 0.8)
    driver.execute_script("window.scrollTo(0, %d)" % top)
    time.sleep(0.5)

  #１秒待って、スクロール後のページの高さを取得する
  time.sleep(1)
  new_last_height = driver.execute_script("return document.body.scrollHeight")

  #スクロール前後でページの高さに変化がなくなったら無限スクロール終了とみなしてループを抜ける
  if last_height == new_last_height:
    break

  #次のループのスクロール開始位置を設定
  last_top = last_height

In [59]:
def download_image(url, file_path):
  r = requests.get(url, stream=True)

  if r.status_code == 200:
    with open(file_path, "wb") as f:
      f.write(r.content)

soup = BeautifulSoup(driver.page_source, 'html.parser')
a_tags = soup.find_all("a")

url = "https://zukan.pokemon.co.jp"   
save_dir = "../Dataset/pokemonzukan/"

for a_tag in a_tags:
    img_tag = a_tag.find("img")
    # スクロール後に現れるデータに対しての処理
    if img_tag.get("loading") == "lazy":
        url_href = url +  a_tag.get("href")
        driver.execute_script("window.open(arguments[0], '_blank')",url_href)
        
        pokemon_name = img_tag.get("alt")
        
        # 現在のウィンドウハンドルを取得する
        current_window_handle = driver.current_window_handle
        # 新しいウィンドウに切り替える
        driver.switch_to.window(driver.window_handles[-1])
        
        # 新しく開いたページ内のHTMLデータを取得する
        soup2 = BeautifulSoup(driver.page_source, 'html.parser')
        main_tag = soup2.find("main")
        img_tag2 = main_tag.find("img")
        
        if img_tag2.get("alt") == pokemon_name:
            # ファイルを保存
            file_name = "{}.png".format(pokemon_name)
            image_path = os.path.join(save_dir, file_name)
            download_image(url=img_tag2.get("src"), file_path=image_path)
        
        # ここでポケモンデータを集めるスクリプト書きたい。
        # print(pokemon_name)
        # div_tags = main_tag.find_all("div")
        # for div_tag in div_tags:
        #   print(div_tag.h2.small.text)
        
        driver.close()
        # 元のウィンドウに切り替える
        driver.switch_to.window(current_window_handle)
        time.sleep(0.2)
        

AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
# https://taiyosite.com/pokeapi-elementary/

In [ ]:
# soup = BeautifulSoup(driver.page_source, 'html.parser')
# image_tags = soup.find_all("img")



# save_dir = "../Dataset/pokemonzukan/"
# images = []
# for image_tag in image_tags:
#   if image_tag.get("loading") == "lazy":
#     # images.append(image_tag.get("src"))
#     file_name = "{}.png".format(image_tag.get("alt"))
#     image_path = os.path.join(save_dir, file_name)
#     download_image(url=image_tag.get("src"), file_path=image_path)